In [ ]:
import pandas as pd
import datetime 
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_rows', 90)
pd.set_option('display.max_columns', 90)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
plots_folder = '../../data/iplan_eda_plots/'
data_folder = '../../data/'
file = 'iplan_sample.csv'
df=pd.read_csv(data_folder + file,parse_dates=
               ["repository_timestamp",
                "dms_timestamp",
                "start_date_time",
                "end_date_time",
                "realized_start_date_time",
                "realized_end_date_time",
                "decl_to_erp_system_date_time",
                "accept_by_employee_date_time",
                "viewable_for_emp_date_time",
                "deleted_date_time",
                "called_up_date_time",
               "start_date",
               "end_date"])

In [ ]:
df = df[df['end_date_time'] < '2022-02-21 00:00:01']

print(df.shape)

print(f"{df.repository_timestamp.min()} - {df.repository_timestamp.max()}")
# what is the meaning of repository_timestamp? these dates suggest that shift data is modified long before shifts start, and after they finish

print(f"{df.start_date.min()} - {df.start_date.max()}")

print(f"{df.start_date.min().strftime('%A')} - {df.start_date.max().strftime('%A')}")

In [ ]:
df.shift_id = df.shift_id.astype('category')
df.pool_id = df.pool_id.astype('category')
df.employee_id = df.employee_id.astype('category')
df.pool_employee_id = df.pool_employee_id.astype('category')

In [ ]:
# begin analysis

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})

missing_value_df.sort_values('percent_missing', inplace=True,ascending=False)
missing_value_df

### What does a typical iPlan week look like?

- number of shifts
- number of employee
- number of pools
    - employees per pool
- average shifts per pool
- average employees per pool
- average employees per shift

In [ ]:
print('Statistics for one week of sample data')
print(f'number of shifts: {df.shift_id.nunique()}')
print(f'number of employees: {df.employee_id.nunique()}')
print(f'number of pools: {df.pool_id.nunique()}')

In [ ]:
test=df.sample(10)
test

In [ ]:
t2 = test[['pool_id','shift_id','employee_id','accept_by_employee_date_time','realized_start_date_time','confirmed_date_time']].groupby(['pool_id'], observed=True).agg({ 'shift_id':'nunique',
                                                                                                           'employee_id':'nunique',
                                                                                                          'accept_by_employee_date_time':'count',
                                                                                                          'realized_start_date_time':'count'})
t2

In [ ]:
t2.columns = ['num_shifts','num_employees','accepted_shifts','realised_shifts']
t2['pct_accepted'] = t2['accepted_shifts'] / t2['num_shifts'] * 100
t2['pct_realised'] = t2['realised_shifts'] / t2['num_shifts'] * 100
t2

#### Pools

dist plot number of employees
dist plot number of shifts
average per day
average per day (mon-friday)

In [ ]:
pooldf = df[['pool_id','shift_id','employee_id','accept_by_employee_date_time','realized_start_date_time','confirmed_date_time']].groupby(['pool_id']).agg({ 'shift_id':'nunique',
                                                                                                           'employee_id':'nunique',
                                                                                                           'accept_by_employee_date_time':'count',
                                                                                                           'realized_start_date_time':'count',
                                                                                                           'confirmed_date_time':'count'})
pooldf.columns = ['num_shifts','num_employees','accepted_shifts','realised_shifts','confirmed_shifts']
pooldf['pct_accepted'] = pooldf['accepted_shifts'] / pooldf['num_shifts'] * 100
pooldf['pct_realised'] = pooldf['realised_shifts'] / pooldf['num_shifts'] * 100
pooldf['pct_confirmed'] = pooldf['confirmed_shifts'] / pooldf['num_shifts'] * 100
pooldf['avg_shifts_pd'] = pooldf['num_shifts'] / 7
pooldf['shift_employee_ratio'] = (pooldf['num_employees'] / pooldf['num_shifts']) * 100
print(pooldf['avg_shifts_pd'].mean())
print(pooldf['shift_employee_ratio'].mean())

In [ ]:
print(len(pooldf[pooldf['num_shifts'] > 500]))
print(len(pooldf[pooldf['num_employees'] > 1000]))

In [ ]:
# remove outliers
pooldf = pooldf[pooldf['num_shifts'] < 500]
pooldf = pooldf[pooldf['num_employees'] < 1000]

In [ ]:
print(f"Shifts per pool: min: {pooldf['num_shifts'].min()}, max: {pooldf['num_shifts'].max()}, mean: {pooldf['num_shifts'].mean()}, median: {pooldf['num_shifts'].median()}")
print(f"Employees per pool: min: {pooldf['num_employees'].min()}, max: {pooldf['num_employees'].max()}, mean: {pooldf['num_employees'].mean()}, median: {pooldf['num_employees'].median()}")

In [ ]:
pooldf['avg_shifts_pd'].mean()

In [ ]:
#define plotting region (2 rows, 2 columns)
sns.set(font_scale=1.5)
#fig, axes = plt.subplots(1, 3,figsize=(20,5))
#

fig = plt.figure(figsize=(20,5))
fig.supylabel('Density')
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)

#create boxplot in each subplot
sns.distplot(pooldf['num_shifts'], color='g', bins=10, ax=ax1).set(ylabel=None)
ax1.set_title('Distribution of Shifts per Pool')
sns.distplot(pooldf['num_employees'], color='g', bins=10, ax=ax2).set(ylabel=None)
ax2.set_title('Distribution of Employees per Pool')
sns.distplot(pooldf['avg_shifts_pd'], color='g', bins=10, ax=ax3).set(ylabel=None)
ax3.set_title('Distribution of Average Shifts per Day')

#save = fig.get_figure()
#fig.savefig(plots_folder+"Pool Distributions.png") 


In [ ]:
fig = plt.figure(figsize=(9, 8))
ax=sns.scatterplot(data=pooldf, x="num_shifts", y="num_employees").set_title('Employees per Pool vs Shifts per Pool')
#fig.savefig(plots_folder+"emplys_pr_pl_v_shfts_pr_pl.png") 

In [ ]:
fig = plt.figure(figsize=(9, 8))
ax=sns.distplot(pooldf['pct_accepted'], color='g', bins=10)
#fig.savefig(plots_folder+"emplys_pr_pl_v_shfts_pr_pl.png") 


#### A closer look into small pools

In [16]:
len(pooldf[pooldf['avg_shifts_pd']<7].sort_values(['num_shifts','pct_accepted'],ascending=[True,False]))

169

In [17]:
simpool = pooldf[pooldf['avg_shifts_pd']<7].sort_values(['num_shifts','pct_accepted'],ascending=[True,False])
simpool.describe()

,num_shifts,num_employees,accepted_shifts,realised_shifts,confirmed_shifts,pct_accepted,pct_realised,pct_confirmed,avg_shifts_pd,shift_employee_ratio
count,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000
mean,23.840237,25.118343,54.011834,29.822485,0.804734,223.781625,122.219133,3.324327,3.405748,107.710813
std,13.267550,35.340489,69.432049,51.820241,3.126839,312.299926,232.855151,13.065417,1.895364,148.641613
min,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.285714,20.000000
25%,12.000000,8.000000,13.000000,0.000000,0.000000,87.500000,0.000000,0.000000,1.714286,42.857143
50%,22.000000,15.000000,32.000000,3.000000,0.000000,148.387097,34.782609,0.000000,3.142857,63.829787
75%,35.000000,29.000000,69.000000,41.000000,0.000000,256.756757,170.000000,0.000000,5.000000,105.405405
max,48.000000,290.000000,540.000000,378.000000,25.000000,2842.105263,1890.000000,94.736842,6.857143,1526.315789


In [18]:
print(len(pooldf[(pooldf['avg_shifts_pd']<7)]))
print(len(pooldf[(pooldf['avg_shifts_pd']<7)&(pooldf['pct_accepted']>100)]))
print(len(pooldf[(pooldf['avg_shifts_pd']<7)&(pooldf['pct_realised']>100)]))
print(len(pooldf[(pooldf['avg_shifts_pd']<7)&(pooldf['pct_confirmed']>50)]))
print(len(pooldf[(pooldf['avg_shifts_pd']<7)&(pooldf['pct_confirmed']==0)]))

169
108
67
4
144


In [19]:
pooldf[(pooldf['avg_shifts_pd']>2)&(pooldf['avg_shifts_pd']<7)].sort_values(['num_shifts','pct_accepted'],ascending=[True,False]).head(20)

,num_shifts,num_employees,accepted_shifts,realised_shifts,confirmed_shifts,pct_accepted,pct_realised,pct_confirmed,avg_shifts_pd,shift_employee_ratio
pool_id,,,,,,,,,,
152.0,15,58,123,188,0,820.000000,1253.333333,0.000000,2.142857,386.666667
10242.0,15,7,29,33,0,193.333333,220.000000,0.000000,2.142857,46.666667
902.0,15,12,22,34,0,146.666667,226.666667,0.000000,2.142857,80.000000
13122.0,15,7,19,25,0,126.666667,166.666667,0.000000,2.142857,46.666667
21066.0,15,4,15,15,0,100.000000,100.000000,0.000000,2.142857,26.666667
749.0,16,45,58,44,1,362.500000,275.000000,6.250000,2.285714,281.250000
14763.0,16,5,2,0,0,12.500000,0.000000,0.000000,2.285714,31.250000
9582.0,17,6,16,19,0,94.117647,111.764706,0.000000,2.428571,35.294118
6358.0,17,8,2,0,0,11.764706,0.000000,0.000000,2.428571,47.058824


In [20]:
pooldf[pooldf['avg_shifts_pd']<7].sort_values(['num_shifts','pct_accepted'],ascending=[True,False]).head(10).to_csv(data_folder+"simple_pools_sample.csv",index=False)

In [ ]:
df[df['pool_id']==17983.0]

In [21]:
twentynine = df[df['pool_id'] == 29.0]
twentynine[["shift_id","shift_day_of_week"]].groupby('shift_day_of_week')['shift_id'].nunique()

shift_day_of_week
Friday        7
Monday        9
Saturday      2
Thursday     10
Tuesday       6
Wednesday     7
Name: shift_id, dtype: int64

In [ ]:
print(twentynine.employee_id.nunique())
print(twentynine.shift_id.nunique())

In [22]:
one = df[df['pool_id'] == 1.0]
one[["shift_id","shift_day_of_week"]].groupby('shift_day_of_week')['shift_id'].nunique()

shift_day_of_week
Friday       4
Monday       2
Thursday     3
Tuesday      2
Wednesday    2
Name: shift_id, dtype: int64

In [33]:
print(one.employee_id.nunique())
print(one.shift_id.nunique()) 

8
13


In [38]:
one = one[one['shift_id'] !=157705104.0] # df[(df['pool_id']==1)] #) & ()]

In [39]:
one.to_csv(data_folder + "simplepool#1.csv",index=False)

In [28]:
onedf = one[['repository_timestamp','shift_id','start_date_time','end_date_time','start_time','end_time','start_date','end_date', \
            'employee_id']].sort_values('start_date_time')
onedf.drop_duplicates(subset=['shift_id'])
len(onedf)

33

### Shifts

In [ ]:
dist plot number of employees
per day?
common start times

In [ ]:
duration outliers

### Employees

In [ ]:
#outliers
outliers=[]
outliers.extend(pooldf.index[pooldf['num_shifts'] > 500].tolist())
outliers.extend(pooldf.index[pooldf['num_employees'] > 1000].tolist())

In [ ]:
dist plot number of shifts
are any in more than one pool?

accept_by_employee_date_time

In [ ]:
len(outliers)

In [ ]:
df[~df['pool_id'].isin(outliers)].shape

In [ ]:
empdf = df[~df['pool_id'].isin(outliers)]
empdf = empdf[['pool_id','shift_id','employee_id','accept_by_employee_date_time']].groupby(['employee_id']).agg({'shift_id' : 'nunique', 
                                                                                                              'pool_id' : 'nunique',
                                                                                                             'accept_by_employee_date_time' : 'count'})

empdf.columns = ['num_shifts','num_pools','num_accepted']

In [ ]:
empdf.head()

In [ ]:
print(f"Shifts per employee: min: {empdf['num_shifts'].min()}, max: {empdf['num_shifts'].max()}, mean: {empdf['num_shifts'].mean()}, median: {empdf['num_shifts'].median()}")
print(f"Accepted shifts per employee: min: {empdf['num_accepted'].min()}, max: {empdf['num_accepted'].max()}, mean: {empdf['num_accepted'].mean()}, median: {empdf['num_accepted'].median()}")
print(f"Accepted shifts per employee: min: {empdf['num_pools'].min()}, max: {empdf['num_pools'].max()}, mean: {empdf['num_pools'].mean()}, median: {empdf['num_pools'].median()}")


In [ ]:
print(f"Shifts per employee: min: {empdf['num_shifts'].min()}, max: {empdf['num_shifts'].max()}, mean: {empdf['num_shifts'].mean()}, median: {empdf['num_shifts'].median()}")
print(f"Accepted shifts per employee: min: {empdf['num_accepted'].min()}, max: {empdf['num_accepted'].max()}, mean: {empdf['num_accepted'].mean()}, median: {empdf['num_accepted'].median()}")
print(f"Accepted shifts per employee: min: {empdf['num_pools'].min()}, max: {empdf['num_pools'].max()}, mean: {empdf['num_pools'].mean()}, median: {empdf['num_pools'].median()}")


In [ ]:
print(f"Number of employees in more than 1 pool: {len(empdf[empdf['num_pools']>1])} ({(len(empdf[empdf['num_pools']>1])/len(empdf))*100}%)")

In [ ]:
empdf[empdf['num_accepted'] > 10].sort_values(['num_accepted'],ascending=False)

In [ ]:
#df[df['pool_id']==225.0]

In [ ]:
df[df['employee_id']==1047368.0].sort_values('start_date_time')

In [ ]:
df.head()